# Deploying the model to an endpoint
In this notebook we will deploy an endpoint with the model whisper-large-v2. We will write our inference code because we will use Whisper's API rather than the Hugging Face API. This is because the Whisper API allows for transcriptions longer than 30 seconds out of the box. The API can be found here: https://github.com/openai/whisper#python-usage

In [4]:
!mkdir model
!mkdir model/code

mkdir: cannot create directory ‘model’: File exists
mkdir: cannot create directory ‘model/code’: File exists


We write out custome inference code here

In [5]:
%%writefile model/code/inference.py
import whisper


def model_fn(model_dir):
    model = whisper.load_model("large-v2")
    return model


def predict_fn(audio_bytes, model):
    audio_file = "tmp.mp3"
    
    with open(audio_file, "wb") as binary_file:
        binary_file.write(audio_bytes['inputs'])
        
    result = model.transcribe(audio_file)

    # print the recognized text
    return {"detected_language": result["language"], "transcription": result["text"]}

Overwriting model/code/inference.py


And into the `requirements.txt` we put the libraries we will need to run the inference code

In [6]:
%%writefile model/code/requirements.txt
git+https://github.com/openai/whisper.git

Overwriting model/code/requirements.txt


## Uploading the model to S3

In [7]:
%cd model

/home/ec2-user/SageMaker/whisper-demo-sagemaker/sagemaker/model


In [8]:
!tar zcvf model.tar.gz *

code/
code/requirements.txt
code/inference.py
model.tar.gz


In [9]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [10]:
role

'arn:aws:iam::905847418383:role/service-role/AmazonSageMaker-ExecutionRole-20210804T091905'

In [11]:
s3_location = f"s3://{sagemaker_session_bucket}/whisper/model/model.tar.gz"

In [12]:
!aws s3 cp model.tar.gz $s3_location

upload: ./model.tar.gz to s3://sagemaker-us-east-1-905847418383/whisper/model/model.tar.gz


## Deplying the model to en endpoint

In [13]:
from sagemaker.huggingface.model import HuggingFaceModel
# from sagemaker.utils import name_from_base
from sagemaker.serializers import DataSerializer

huggingface_model = HuggingFaceModel(
   model_data=s3_location,
   role=role,
   transformers_version="4.17",
   pytorch_version="1.10",
   py_version='py38',
)

In [14]:
endpoint_name = "whisper-large-v2"
audio_serializer = DataSerializer(content_type='audio/x-audio')

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    endpoint_name=endpoint_name,
    serializer=audio_serializer,
)

----------!

In [15]:
%cd ..

/home/ec2-user/SageMaker/whisper-demo-sagemaker/sagemaker


In [16]:
audio_path = "mpthreetest.mp3"

res = predictor.predict(data=audio_path)
print(res)

{'detected_language': 'en', 'transcription': ' Okay, we are trying this for a second time to test the ability to upload an MP3 file. Hopefully this will work.'}
